# Basic Recipe: Static Analysis of a Simple Reform

This is the recipe you should follow first.
Mastering this recipe is a prerequisite for all the other recipes in this cookbook.

## Imports

In [ ]:
# Install conda and taxcalc if in Google Colab.
import sys
if 'google.colab' in sys.modules and 'taxcalc' not in sys.modules:
    !wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
    !bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
    # Append path to be able to run packages installed with conda
    # This must correspond to the conda Python version, which may differ from
    # the base Colab Python installation.
    sys.path.append('/usr/local/lib/python3.7/site-packages')
    # Install PSL packages from Anaconda
    !yes | conda install -c PSLmodels taxcalc behresp

In [1]:
import taxcalc as tc
import pandas as pd
from bokeh.io import show, output_notebook

## Setup

Use publicly-available CPS input file.

NOTE: if you have access to the restricted-use IRS-SOI PUF-based input file
and you have that file (named ‘puf.csv’) located in the directory
where this script is located, then you can substitute the following
statement for the prior statement:

``
recs = tc.Records()
``

In [2]:
recs = tc.Records.cps_constructor()

Specify Calculator object for static analysis of current-law policy.

In [3]:
pol = tc.Policy()
calc1 = tc.Calculator(policy=pol, records=recs)

NOTE: calc1 now contains a PRIVATE COPY of pol and a PRIVATE COPY of recs,
so we can continue to use pol and recs in this script without any
concern about side effects from Calculator method calls on calc1.

In [4]:
CYR = 2020

Calculate aggregate current-law income tax liabilities for CYR.

In [5]:
calc1.advance_to_year(CYR)
calc1.calc_all()
itax_rev1 = calc1.weighted_total('iitax')

Read JSON reform file and use (the default) static analysis assumptions.

In [6]:
reform_filename = '_static/reformA.json'
params = tc.Calculator.read_json_param_objects(reform_filename, None)

Specify Calculator object for static analysis of reform policy.

In [7]:
pol.implement_reform(params['policy'])
calc2 = tc.Calculator(policy=pol, records=recs)

## Calculate

Calculate reform income tax liabilities for CYR.

In [8]:
calc2.advance_to_year(CYR)
calc2.calc_all()
itax_rev2 = calc2.weighted_total('iitax')

## Results

Print total revenue estimates for 2018.

*Estimates in billons of dollars rounded to nearest hundredth of a billion.*

In [9]:
print('{}_CLP_itax_rev($B)= {:.3f}'.format(CYR, itax_rev1 * 1e-9))
print('{}_REF_itax_rev($B)= {:.3f}'.format(CYR, itax_rev2 * 1e-9))

2020_CLP_itax_rev($B)= 1430.743
2020_REF_itax_rev($B)= 1423.661


Generate several other standard results tables.

In [10]:
# Aggregate diagnostic tables for CYR.
clp_diagnostic_table = calc1.diagnostic_table(1)
ref_diagnostic_table = calc2.diagnostic_table(1)

# Income-tax distribution for CYR with CLP and REF results side-by-side.
dist_table1, dist_table2 = calc1.distribution_tables(calc2, 'weighted_deciles')
assert isinstance(dist_table1, pd.DataFrame)
assert isinstance(dist_table2, pd.DataFrame)
dist_extract = pd.DataFrame()
dist_extract['funits(#m)'] = dist_table1['count']
dist_extract['itax1($b)'] = dist_table1['iitax']
dist_extract['itax2($b)'] = dist_table2['iitax']
dist_extract['aftertax_inc1($b)'] = dist_table1['aftertax_income']
dist_extract['aftertax_inc2($b)'] = dist_table2['aftertax_income']

# Income-tax difference table by expanded-income decile for CYR.
diff_table = calc1.difference_table(calc2, 'weighted_deciles', 'iitax')
assert isinstance(diff_table, pd.DataFrame)
diff_extract = pd.DataFrame()
dif_colnames = ['count', 'tot_change', 'mean', 'pc_aftertaxinc']
ext_colnames = ['funits(#m)', 'agg_diff($b)', 'mean_diff($)', 'aftertaxinc_diff(%)']
for dname, ename in zip(dif_colnames, ext_colnames):
    diff_extract[ename] = diff_table[dname]

## Plotting

Generate a decile graph and display it using Bokeh (will render in Jupyter, not in webpage).

In [11]:
fig = calc1.pch_graph(calc2)
output_notebook()
show(fig)

Loading BokehJS ...

## Print tables

CLP diagnostic table for CYR.

In [12]:
clp_diagnostic_table

,2020
Returns (#m),167.510
AGI ($b),12081.707
Itemizers (#m),33.100
Itemized Deduction ($b),935.939
Standard Deduction Filers (#m),134.410
Standard Deduction ($b),2382.421
Personal Exemption ($b),0.000
Taxable Income ($b),9264.215
Regular Tax ($b),1588.955
AMT Income ($b),11415.741


REF diagnostic table for CYR.

In [13]:
ref_diagnostic_table

,2020
Returns (#m),167.510
AGI ($b),12081.707
Itemizers (#m),33.030
Itemized Deduction ($b),933.241
Standard Deduction Filers (#m),134.480
Standard Deduction ($b),2383.809
Personal Exemption ($b),333.334
Taxable Income ($b),9011.004
Regular Tax ($b),1579.369
AMT Income ($b),11418.092


Extract of CYR distribution tables by baseline expanded-income decile.

In [14]:
dist_extract

,funits(#m),itax1($b),itax2($b),aftertax_inc1($b),aftertax_inc2($b)
0-10n,0.05,0.006,0.006,-14.549,-14.549
0-10z,0.93,0.000,0.000,-0.000,-0.000
0-10p,15.77,-4.022,-4.393,176.755,177.126
10-20,16.75,-1.134,-2.213,416.243,417.322
20-30,16.75,4.097,2.854,554.417,555.660
30-40,16.75,10.684,8.977,683.213,684.920
40-50,16.75,20.429,18.028,841.306,843.707
50-60,16.75,34.373,31.268,1034.418,1037.523
60-70,16.75,63.586,59.589,1273.366,1277.363
70-80,16.75,110.120,104.697,1598.678,1604.101


Extract of CYR income-tax difference table by expanded-income decile.

In [15]:
diff_extract

,funits(#m),agg_diff($b),mean_diff($),aftertaxinc_diff(%)
0-10n,0.05,0.000,1.5,0.0
0-10z,0.93,0.000,0.0,0.0
0-10p,15.77,-0.371,-23.5,0.2
10-20,16.75,-1.078,-64.4,0.3
20-30,16.75,-1.243,-74.2,0.2
30-40,16.75,-1.706,-101.9,0.2
40-50,16.75,-2.401,-143.3,0.3
50-60,16.75,-3.105,-185.4,0.3
60-70,16.75,-3.997,-238.6,0.3
70-80,16.75,-5.423,-323.8,0.3
